In [1]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
train_data = pd.read_csv('F:/data/train_2018Q1_model_2.csv')

In [3]:
class MyData(Dataset):
    def __init__(self, train_data):
        self.value = train_data.iloc[:, 6:-1].values
        max = np.max(self.value)
        min = np.min(self.value) 
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.label = train_data.loc[:, 'label'].values.tolist()
    def __len__(self):
        return len(self.datas)
    def __getitem__(self, idx):
        data = self.datas[idx].reshape(1, -1)
        return data, self.label[idx]


train_iter = DataLoader(MyData(train_data), batch_size = 10)

In [4]:
for X, y in train_iter:
    print(X.shape)
    print(X)
    break

torch.Size([10, 1, 13])
tensor([[[1.0101, 1.0101, 1.0101, 0.7980, 1.0101, 0.2121, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.7980, 1.0101, 0.2121, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.8081, 1.0101, 0.2020, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.7980, 1.0101, 0.2121, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.7980, 1.0101, 0.2121, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.7980, 1.0101, 0.2121, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.8081, 1.0101, 0.2020, 1.0101, 1.0101,
          1.0101, 1.0101, 0.8586, 1.0101, 0.6970]],

        [[1.0101, 1.0101, 1.0101, 0.8081, 1.0101, 0.2020, 1.0101, 1.0101,
          1.0

In [5]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



def mySample(data):
    num = 0
    total = len(train_set)
    sample_list = []

    for X, y in train_set:
        num = num + y
    

    class_sample_count = np.array([total - num, num])
    weight = 1. / class_sample_count

    for X, y in train_set:
        sample_list.append(weight[int(y)])

    sample_weight = torch.FloatTensor(sample_list)
    sampler = WeightedRandomSampler(sample_weight, total)
    return sampler

class MyData(Dataset):
    def __init__(self, traindata, numstep, length):
        self.serial_number = traindata['serial_number'].value_counts()
        self.value = traindata.iloc[:, length:-1].values
        max = np.max(self.value)
        min = np.min(self.value)
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.datalabel = train_data.loc[:, 'label'].values.tolist()
        self.input = []
        self.label = []

        for diskname in self.serial_number.index.sort_values():
            traindata_name = traindata[traindata.serial_number == diskname]
            self.datalabel = traindata_name.loc[:, 'label'].values.tolist()
            for i in range(len(traindata_name) - numstep):
                self.input.append(torch.Tensor(self.datas[i: i + numstep]))
                if(torch.Tensor(self.datalabel[i: i + numstep]).sum() != 0):
                    self.label.append(torch.ones(1))
                else:
                    self.label.append(torch.zeros(1))
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        #data = self.datas[idx].reshape(1, -1)
        return self.input[idx], self.label[idx]

train_data = pd.read_csv('F:/data/train_2018Q1_model_2.csv')
train_set = MyData(train_data, 7, 6)
sampler = mySample(train_set)
train_iter = DataLoader(MyData(train_data, 7, 6), batch_size = 10, drop_last=True)


In [6]:
class myLSTM(nn.Module):

    def __init__(self):
        super(myLSTM, self).__init__()

        self.lstmlayer = nn.LSTM(input_size=13, hidden_size=128, num_layers=1, batch_first=True)
        
    def forward(self, X):
        X, (H, C) = self.lstmlayer(X)
        return X

In [7]:
for X, y in train_iter:
    print(X.shape)
    print(y.shape)
    print(y.reshape(-1).shape)
    break

torch.Size([10, 7, 13])
torch.Size([10, 1])
torch.Size([10])


In [5]:
net = myLSTM()
y_hat = net(X.float())
y_hat.shape

torch.Size([10, 2])

In [18]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)

In [19]:
target.shape

torch.Size([3])

In [20]:
input.shape

torch.Size([3, 5])

In [9]:
net = myLSTM()

In [11]:
net(X).shape

torch.Size([10, 7, 128])